<a href="https://colab.research.google.com/github/Janindu-Muthunayaka/model-distillation/blob/main/DistillationV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Packages

# Data Preparation

In [16]:
!pip install numpy pandas scikit-learn torch


DataSets (Wine for Classification California Housing for Regression)

In [17]:
from sklearn.datasets import load_wine,fetch_california_housing

#classification  - Wine for type of wine (0,1,2)
wine=load_wine(as_frame=True)
X_Wine=wine.data
Y_Wine=wine.target

#Scaling

from sklearn.preprocessing import StandardScaler

wineScaler=StandardScaler()
XWineScaled=wineScaler.fit_transform(X_Wine)



Teacher MLP Creation


Going to create a fresh teacher MLP

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [22]:
class wineTeacher(nn.Module):
  def __init__(self):
    super(wineTeacher,self).__init__()
    self.fc1=nn.Linear(13,64)
    self.fc2=nn.Linear(64,32)
    self.fc3=nn.Linear(32,16)
    self.fc4=nn.Linear(16,8)
    self.fc5=nn.Linear(8,3)

  def forward(self,x):
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=F.relu(self.fc3(x))
    x=F.relu(self.fc4(x))
    x=self.fc5(x)
    return x

#Student MLP Creation


Going to Create fresh Student MLP

In [23]:
class wineStudent(nn.Module):
  def __init__(self):
    super(wineStudent,self).__init__()
    self.fc1=nn.Linear(13,16)
    self.fc2=nn.Linear(16,8)
    self.fc3=nn.Linear(8,3)

  def forward(self,x):
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=self.fc3(x)
    return x

# Data Preparation for PyTorch

In [25]:
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#split
xTrain,xTest,yTrain,yTest=train_test_split(XWineScaled,Y_Wine,test_size=0.2,random_state=42)

#scaling
scaler=StandardScaler()
xTrain=scaler.fit_transform(xTrain)
xTest=scaler.transform(xTest)

#convert top Pytorch tensors
xTrain=torch.tensor(xTrain,dtype=torch.float32)
yTrain=torch.tensor(yTrain.values,dtype=torch.long)
xTest=torch.tensor(xTest,dtype=torch.float32)
yTest=torch.tensor(yTest.values,dtype=torch.long)






# Training

Training Loop


In [26]:
import torch.optim as optim

teacher = wineTeacher()  # teacher model
criterion = nn.CrossEntropyLoss()  # classification loss
optimizer = optim.Adam(teacher.parameters(), lr=0.001)
epochs = 50  # REMINDER -CHANGE

for epoch in range(epochs):
    teacher.train()
    optimizer.zero_grad()
    outputs = teacher(xTrain)
    loss = criterion(outputs, yTrain)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:#print only every 10 eopich for cleaness
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')


Epoch [10/50], Loss: 1.1006
Epoch [20/50], Loss: 1.0649
Epoch [30/50], Loss: 1.0036
Epoch [40/50], Loss: 0.8885
Epoch [50/50], Loss: 0.7001


# Testing

Accuracy

In [27]:
teacher.eval()
with torch.no_grad():
    y_pred = teacher(xTest)
    predicted_classes = torch.argmax(y_pred, dim=1)
    accuracy = (predicted_classes == yTest).float().mean()
    print(f'Teacher Accuracy: {accuracy:.4f}')


Teacher Accuracy: 0.9444


Inference Time

In [28]:
import torch
import time

teacher.eval()
with torch.no_grad():
    start_time = time.time()
    y_pred = teacher(xTest)
    end_time = time.time()

elapsed_time = end_time - start_time
print(f"Inference time for {xTest.size(0)} samples: {elapsed_time:.6f} seconds")
print(f"Average time per sample: {elapsed_time / xTest.size(0):.6f} seconds")


Inference time for 36 samples: 0.004379 seconds
Average time per sample: 0.000122 seconds


Memory Size

In [32]:
# Number of trainable parameters
num_params = sum(p.numel() for p in teacher.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {num_params}")

# Approximate size in MB (assuming 32-bit float)
param_size_MB = num_params * 4 / (1024**2)  # 4 bytes per float32
print(f"Approximate model size: {param_size_MB:.6f} MB")


Number of trainable parameters: 3667
Approximate model size: 0.013988 MB
